In [22]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [50]:
import os
import traci
import sys
import argparse
import tensorflow as tf
import numpy as np
from dotenv import load_dotenv

# Import internal modules
from rl_package.rl_logic.Environnement import EnvironnementSumo
from rl_package.rl_logic.Agent import AgentSumo
from rl_package.params import *
print(BATCH_SIZE,NAME_SIMULATION)

32 simu_simple


In [51]:


# Load environment variables
load_dotenv()


#SUMO_BIN = r"C:/Program Files/rl_project/Eclipse/Sumo/bin/sumo.exe"
# SIMUL_CONFIG = r"double_traffic/double_traffic.sumo.cfg"
SIMUL_CONFIG
WINDOW=2000
BATCH_SIZE=6
# SUMO command

In [52]:
def preprocess():
    """
    Determines the number of inputs and outputs required for each agent.
    """
    sumoCmd = [SUMO_BIN, "-c", SIMUL_CONFIG, '--start', '--no-warnings']
    env = EnvironnementSumo(sumoCmd, WINDOW)
    inputs_per_agents = []
    outputs_per_agents = []
    positions_phases = []

    for trafficlight in env.trafficlights_ids:
        # Get the number of lanes controlled by this traffic light
        n_lanes = len(env.control_lanes(trafficlight))
        inputs_per_agents.append(n_lanes * 3)  # Inputs: queue + vehicle count

        # Get the number of valid traffic light phases (excluding yellow)
        n_phases,position = env.get_phase_without_yellow(trafficlight)
        #print(f'trafficlight :{trafficlight},\n lane associated {env.control_lanes(trafficlight)}')
        n_outputs = len(n_phases)
        positions_phases.append(position)
        outputs_per_agents.append(n_outputs)

    env.close()
    return inputs_per_agents, outputs_per_agents,positions_phases  # List of inputs, outputs per agent, and the postion phases of each trafficlight


In [53]:
def train_models(inputs_per_agents, outputs_per_agents, position_phases, type_model="DQN"):
    """
    Trains multiple reinforcement learning agents to optimize traffic lights.
    Saves each model separately.
    """
    agents = [AgentSumo(type_model, inputs, outputs) for inputs, outputs in zip(inputs_per_agents, outputs_per_agents)]

    # Load pre-trained models if available
    for i, agent in enumerate(agents):
        agent.build_model()
        model_path = f"models/{NAME_SIMULATION}_{type_model}_Agent{i}.keras"
        if os.path.exists(model_path):
            print(f"🔄 Loading pre-trained model for Agent {i} from {model_path}...")
            agent.model_action = tf.keras.models.load_model(model_path)
            if agent.model_target:  # For Double/Dueling DQN
                agent.model_target = tf.keras.models.load_model(model_path)

    sumoCmd = [SUMO_BIN, "-c", SIMUL_CONFIG, '--start', '--no-warnings']



    for episode in range(EPISODE):
        print(f'🔄 Episode {episode}/{EPISODE}')
        env = EnvironnementSumo(sumoCmd, WINDOW)

        #Store the position phases of the trafficlight in the environment
        env.position_phases = position_phases

        epsilon = max(1 - episode / EPISODE, 0.01)  # Decaying epsilon for exploration

        traffic_lights = env.trafficlights_ids
        states = [env.get_states_per_traffic_light(traffic_light) for traffic_light in traffic_lights]

        for _ in range(50):  # Steps per episode
            actions = [agent.epsilon_greedy_policy(np.array(states[i]), epsilon) for i, agent in enumerate(agents)]
            next_states, rewards = env.step(actions)

            for i in range(len(agents)):
                agents[i].add_to_memory(np.array(states[i]), np.array(actions[i]), np.array(rewards[i]), np.array(next_states[i]))

            states = next_states

            if len(agents[0].replay_buffer) >= BATCH_SIZE *1:
                for agent in agents:
                    agent.training_step(BATCH_SIZE)

            if env.get_total_number_vehicles() == 0:
                break  # Stop simulation if no vehicles left

        # Update target networks every 5 episodes for Double/Dueling DQN
        if episode % 5 == 0 and type_model != 'DQN':
            for agent in agents:
                agent.model_target.set_weights(agent.model_action.get_weights())

        env.close()

    for i, agent in enumerate(agents):
        model_path = f"models/{NAME_SIMULATION}_{type_model}_Agent{i}.keras"
        agent.model_action.save(model_path)
        print(f"✅ Model saved for Agent {i} at: {model_path}")


In [54]:

def load_trained_agents(inputs_per_agents, outputs_per_agents, type_model="DQN"):
    """
    Loads pre-trained agents from saved model files.
    If any model is missing, exits the program.
    """
    agents = [AgentSumo(type_model, inputs, outputs) for inputs, outputs in zip(inputs_per_agents, outputs_per_agents)]

    for i, agent in enumerate(agents):
        model_path = f"models/{NAME_SIMULATION}_{type_model}_Agent{i}.keras"
        if os.path.exists(model_path):
            print(f"🔄 Loading pre-trained model for Agent {i} from {model_path}...")
            agent.build_model()
            agent.model_action = tf.keras.models.load_model(model_path)
        else:
            print(f"❌ No pre-trained model found for Agent {i}.")
            sys.exit(1)

    return agents

In [55]:
def scenario(agents,positions_phases):
    """
    Runs a SUMO simulation using the trained agents.
    """
    sumoCmd = [SUMO_GUI_BIN, "-c", SIMUL_CONFIG, '--start', '--no-warnings']
    env = EnvironnementSumo(sumoCmd, WINDOW)
    #Store the position phases of the trafficlight in the environment
    env.position_phases = positions_phases
    env.full_simul(agents)

In [56]:

type_model = "2DQN"
SIMUL_CONFIG = "/Users/arseneclaustre/code/psels/RL_traffic/Traffic/Traditional_traffic/traditional_traffic.sumo.cfg"
WINDOW=2000
EPISODE = 20
print(NAME_SIMULATION)
inputs_per_agents, outputs_per_agents,positions_phases = preprocess()
print(f'inputs : {inputs_per_agents}')
print(f'outputs : {outputs_per_agents}')
print(f'positions : {positions_phases}')
train_models(inputs_per_agents, outputs_per_agents, positions_phases, type_model)
agents = load_trained_agents(inputs_per_agents, outputs_per_agents, type_model)
scenario(agents,positions_phases)


simu_simple
 Retrying in 1 seconds


Step #0.00 (0ms ?*RT. ?UPS, TraCI: 1ms, vehicles TOT 0 ACT 0 BUF 0)                      
inputs : [36]
outputs : [4]
positions : [[0, 2, 4, 6]]
🚀 Création d'un nouveau modèle 2DQN...
🔄 Episode 0/20
 Retrying in 1 seconds


Step #1000.00 (0ms ?*RT. ?UPS, TraCI: 55ms, vehicles TOT 770 ACT 127 BUF 4)                 ?*RT. ?UPS, TraCI: 57ms, vehicles TOT 559 ACT 147 BUF 5)                
🔄 Episode 1/20
 Retrying in 1 seconds


state [ 0.    0.    0.    0.    1.2   0.    0.    0.    0.    0.    6.99  0.
  0.    0.    0.    0.    2.    0.    0.    0.    0.    0.   15.    0.
  1.    0.   14.    1.    4.    2.    5.    0.    0.    1.   19.    0.  ]
Q_values [-3.0239549 -6.9324307 16.624432  -3.2487533]
state [ 0.    1.14  1.56  0.    0.    0.78  0.    0.    0.    0.    0.    0.
  0.    1.   10.    0.    0.    2.    0.    0.    0.    0.    0.    0.
  0.    2.   19.    2.    2.    4.    4.    0.    0.    1.   11.    0.  ]
Q_values [-3.665717   5.5746155  6.159738  -0.8425819]
Step #720.00 (1ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 558 ACT 64 BUF 0)                  state [ 0.    0.    0.    0.   15.9  20.13  0.    0.    0.    0.    3.67  0.
  0.    0.    0.    0.   17.   29.    0.    0.    0.    0.    4.    0.
  0.    0.    2.    5.   21.   33.    2.    0.    0.    0.    6.    0.  ]
Q_values [  6.6328297 -49.785862    4.0542912 -29.086645 ]
Step #1000.00 (0ms ?*RT. ?UPS, TraCI: 54ms, vehicles TOT 770 ACT 88 BUF 4) 

state [ 0.    0.   65.7   0.    0.21  4.93  0.    0.    9.02  0.    1.63  1.61
  0.    0.   34.    0.    2.    6.    0.    0.    4.    0.    9.    2.
  0.    1.   37.    2.    2.    6.    5.    0.    4.    1.   13.    2.  ]
Q_values [-98.11876   29.008533  76.746704  38.885384]
state [ 0.    0.   79.86  0.    0.    7.33  0.    0.   10.62  0.    0.    2.41
  0.    0.   37.    0.    0.    6.    0.    0.    4.    0.    0.    2.
  0.    0.   40.    2.    2.    8.    4.    0.    4.    0.    4.    2.  ]
Q_values [-84.21031  30.75431  65.25969  40.42184]
state [ 0.    5.14  1.64  0.    3.53  0.    0.    5.36  0.    0.   25.61  0.
  0.    2.   10.    0.    4.    0.    0.    2.    0.    0.   25.    0.
  2.    4.   22.    2.    6.    2.    4.    4.    2.    2.   29.    2.  ]
Q_values [-18.843014  -9.29239   45.570267 -11.15383 ]
Step #1000.00 (0ms ?*RT. ?UPS, TraCI: 55ms, vehicles TOT 770 ACT 92 BUF 4)                  ?*RT. ?UPS, TraCI: 56ms, vehicles TOT 559 ACT 78 BUF 5)                 
🔄 Ep

...

state [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Q_values [-8.303919    0.08967936 -0.9476974  -2.8369637 ]
state [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 2. 3. 3. 2. 2. 2. 4. 2. 2. 3. 3. 2.]
Q_values [-8.254604   -1.387198    0.13569598 -1.4097173 ]
state [0.   0.24 0.14 0.   0.   0.17 0.   0.23 0.16 0.   0.   0.15 0.   2.
 2.   0.   0.   2.   0.   2.   2.   0.   0.   2.   1.   2.   7.   2.
 0.   2.   5.   2.   2.   1.   3.   2.  ]
Q_values [-8.275797   -1.3844503   0.12280797 -1.4085317 ]
state [ 0.    1.04  1.96  0.    0.    0.97  0.    1.03  0.96  0.    0.    0.95
  0.    2.    6.    0.    0.    2.    0.    2.    2.    0.    0.    2.
  0.    2.   10.    2.    2.    4.    4.    2.    2.    0.    3.    2.  ]
Q_values [-8.300935   -1.3827269   0.11373731 -1.4098217 ]
state [ 0.    0.    9.76  0.    0.    3.75  0.    0.    2.56  0.    0.45  2.55
  0.    0.   13.    0.    0.    4.    0. 

state [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Q_values [-9.73066    -1.8000203   0.40211725 -2.8132634 ]
state [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 2. 3. 3. 2. 2. 2. 4. 2. 2. 3. 3. 2.]
Q_values [-9.136079   -1.5484407  -0.41343546 -1.4549747 ]
state [ 0.    1.04  1.96  0.    0.    0.97  0.    1.03  0.96  0.    0.    0.95
  0.    2.    6.    0.    0.    2.    0.    2.    2.    0.    0.    2.
  0.    2.   10.    2.    2.    4.    4.    2.    2.    0.    3.    2.  ]
Q_values [-9.118416  -1.5211229 -0.4794327 -1.4571103]
state [ 0.    1.84  5.16  0.    0.    2.15  0.    1.83  1.76  0.    0.    1.75
  0.    2.   10.    0.    0.    4.    0.    2.    2.    0.    0.    2.
  0.    3.   13.    4.    1.    4.    4.    2.    2.    1.    3.    2.  ]
Q_values [-9.114027   -1.5151635  -0.49116927 -1.4509572 ]
state [ 0.    3.44 15.71  0.    0.29  0.    0.    3.43  3.36  0.    0.24  0.
  0.    2

state [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Q_values [-13.287699    -0.17735267  -0.91786593  -2.368509  ]
state [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 2. 3. 3. 2. 2. 2. 4. 2. 2. 3. 3. 2.]
Q_values [-10.337878   -1.2069938  -0.7938676  -1.5369831]
state [0.   0.24 0.14 0.   0.   0.17 0.   0.23 0.16 0.   0.   0.15 0.   2.
 2.   0.   0.   2.   0.   2.   2.   0.   0.   2.   1.   2.   7.   2.
 0.   2.   5.   2.   2.   1.   3.   2.  ]
Q_values [-10.358738   -1.211168   -0.8015498  -1.5525843]
state [ 0.    1.04  1.96  0.    0.    0.97  0.    1.03  0.96  0.    0.    0.95
  0.    2.    6.    0.    0.    2.    0.    2.    2.    0.    0.    2.
  0.    2.   10.    2.    2.    4.    4.    2.    2.    0.    3.    2.  ]
Q_values [-10.383229   -1.2224911  -0.8084696  -1.5608813]
state [ 0.    1.84  5.16  0.    0.    2.15  0.    1.83  1.76  0.    0.    1.75
  0.    2.   10.    0.    0.    4.   

state [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Q_values [-16.577765    4.2416553  -3.00405    -2.9338732]
state [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 2. 3. 3. 2. 2. 2. 4. 2. 2. 3. 3. 2.]
Q_values [-10.83073    -1.0658901  -1.2675133  -1.763378 ]
state [0.   0.24 0.   0.   0.22 0.17 0.   0.23 0.   0.   0.26 0.15 0.   2.
 0.   0.   2.   2.   0.   2.   0.   0.   3.   2.   1.   2.   3.   2.
 2.   2.   6.   2.   0.   1.   7.   2.  ]
Q_values [-10.838848   -1.0569036  -1.305321   -1.7692446]
state [ 0.    1.04  0.    0.    1.02  0.97  0.    1.03  0.    0.    2.09  0.95
  0.    2.    0.    0.    2.    2.    0.    2.    0.    0.    6.    2.
  0.    2.    3.    2.    4.    4.    4.    2.    0.    0.   10.    2.  ]
Q_values [-10.846119   -1.0496681  -1.3489481  -1.7745287]
state [ 0.    1.84  0.    0.    2.28  2.15  0.    1.83  0.    0.    5.3   1.75
  0.    2.    0.    0.    4.    4.    0. 

FatalTraCIError: Connection closed by SUMO.

In [ ]:
print(NAME_SIMULATION)

simu_simple


In [58]:
scenario(agents,positions_phases)

 Retrying in 1 seconds
state [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Q_values [-17.03658     6.665996   -4.0581965  -4.565104 ]
state [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 2. 3. 3. 2. 2. 2. 4. 2. 2. 3. 3. 2.]
Q_values [-11.199076   -1.2317432  -1.6289366  -1.6423672]
state [0.   0.24 0.   0.   0.22 0.17 0.   0.23 0.   0.   0.26 0.15 0.   2.
 0.   0.   2.   2.   0.   2.   0.   0.   3.   2.   1.   2.   3.   2.
 2.   2.   6.   2.   0.   1.   7.   2.  ]
Q_values [-11.199076   -1.2317432  -1.6289366  -1.6423672]
state [ 0.    1.04  0.    0.    1.02  0.97  0.    1.03  0.    0.    2.09  0.95
  0.    2.    0.    0.    2.    2.    0.    2.    0.    0.    6.    2.
  0.    2.    3.    2.    4.    4.    4.    2.    0.    0.   10.    2.  ]
Q_values [-11.199076   -1.2317432  -1.6289366  -1.6423672]
state [ 0.    1.84  0.    0.    2.28  2.15  0.    1.83  0.    0.    5.3   1.75
  0.    2.    0.  

FatalTraCIError: Connection closed by SUMO.